# Charging type identification
We want to know how the vehicles are being charged.  
To do so, we will either use the explicit data provided by the data providers or try to infer from other variables such as the speed of the charging or the power of the charging.

## Setup

### Imports

In [ ]:
import plotly.express as px

from core.pandas_utils import *
from transform.raw_tss.main import get_raw_tss, GET_RAW_TSS_FUNCTIONS
from transform.processed_tss.main import get_processed_tss, GET_PROCESSED_TSS_FUNCTIONS

### Raw time series EDA

We will look at the sanity checks of each brand to find the columns that we can use to infer the charging type.  
Currently, most of these variables are not passed into processed_tss so we have to look into raw_tss.  

In [ ]:
for brand in GET_RAW_TSS_FUNCTIONS.keys():
    print(brand)
    processed_tss = get_raw_tss(brand).filter(like="charg")
    display(sanity_check(processed_tss))
    print("============================")

### Processed time series EDA

Here is what we can use for each brand:  
- bmw: charging_method  

- tesla: fast_charger_type, fast_charger_present, charge_rate, charger_actual_current, charger_power, charging_state  
    We could also use charge_port_cold_weather_mode, charger_pilot_current, charger_voltage  

- kia: There does not seem to be any explicit charging type data we might have to infer it from the charging speed.  

- mercedes-benz: charging.smart_charging_status, charging.status, charging.charging_rate  

- ford: No explicit charging type data. We might have to infer it from the charging speed.  

- renault:charging.status, charging.battery_charge_type, charging.charging_rate  

- opel: electricity.charging.mode, electricity.charging.planned, electricity.charging.rate  

- peugeot: electricity.charging.mode, electricity.charging.planned, electricity.charging.rate

- ds: electricity.charging.mode, electricity.charging.rate

- fiat: No explicit charging type data. We might have to infer it from the charging speed.  

- volvo-cars: charging.status maybe? Otherwise we might have to infer it from the charging speed.  

We will visualize the various in e time series scatter plots to compare the the different values of the charging type.  

In [ ]:
for brand in GET_PROCESSED_TSS_FUNCTIONS.keys():
    print(brand)
    processed_tss = get_processed_tss(brand)
    if "charging_method" in processed_tss.columns:
        most_common_vin = processed_tss["vin"].value_counts().idxmax()
        ts:DF = (
            processed_tss
            .query("vin == @most_common_vin")
            .astype({"charging_method": "string"})
            .eval("charging_method = charging_method.ffill()")
            .dropna(subset=["soc", "date"])
        )
        display(ts["charging_method"].value_counts())
        px.scatter(
            ts,
            x="date",
            y="soc",
            color="charging_method",
            title=f"{brand}, {most_common_vin} charging method",
        ).show()
    else:
        print("No charging method column.")
    print("============================")


### Charging power
Looking at the various brands we can see that we cannot get the charging types for all brands using only the charging_method column.  
This is because a few brands simply don't have it and because the ones that do have it, the charging methods are not always informative.  
Let's try to use the charging power instead.

In [ ]:
ts.columns

In [ ]:
opel_tss = get_raw_tss("opel")
most_common_vin = opel_tss["vin"].value_counts(ascending=False, sort=True).idxmax()
ts:DF = (
    opel_tss
    .query("vin == @most_common_vin")
    .eval("`electricity.charging.rate` = `electricity.charging.rate`.ffill()")

)
px.scatter(
    ts.dropna(subset=["electricity.level.percentage", "datetime"]),
    x="datetime",
    y="electricity.level.percentage",
    color="electricity.charging.rate"
)